In [2]:
import cv2
from imutils import paths
import numpy as np
import math
from tqdm import tqdm
import os
from PIL import Image

In [2]:
def calculate_psnr(img1_tensor, img2_tensor):
    """Calculate PSNR score
    
    Parameters:
        img1_tensor (int) -- a image to compare, range [0, 255]
        img2_tensor (int) -- a image to compare, range [0, 255]
    """    
    img1 = ((img1_tensor + 1.0)*255.0/2.0).astype(np.float64)
    img2 = ((img2_tensor + 1.0)*255.0/2.0).astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

def calculate_ssim(img1_tensor, img2_tensor):
    """Calculate SSIM score
    
    Parameters:
        img1_tensor (int) -- a image to compare, range [0, 255]
        img2_tensor (int) -- a image to compare, range [0, 255]
    """
    def ssim(img1, img2):
        C1 = (0.01 * 255)**2
        C2 = (0.03 * 255)**2

        kernel = cv2.getGaussianKernel(11, 1.5)
        window = np.outer(kernel, kernel.transpose())

        mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
        mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
        mu1_sq = mu1**2
        mu2_sq = mu2**2
        mu1_mu2 = mu1 * mu2
        sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
        sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
        sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

        ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                                (sigma1_sq + sigma2_sq + C2))
        return ssim_map.mean()
    
    img1 = ((img1_tensor + 1.0)*255.0/2.0).astype(np.float64)
    img2 = ((img2_tensor + 1.0)*255.0/2.0).astype(np.float64)
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

In [3]:
groudtruth = list(paths.list_images("C:\\Users\\lemin\\Downloads\\result_set\\groudtruth__vein"))
original = list(paths.list_images("C:\\Users\\lemin\\Downloads\\result_set\\original__vein"))

PNSR_dict, SSIM_dict, PNSR_dict_in, SSIM_dict_in, PNSR_dict_out, SSIM_dict_out = 0, 0, 0, 0, 0, 0
for i in tqdm(range(len(original))):
    img_original = (cv2.imread(original[i])/255.0-0.5)*2.0
    img_groudtruth = (cv2.imread(groudtruth[i])/255.0-0.5)*2.0
    
    PNSR_dict += calculate_psnr(img_groudtruth, img_original)
    SSIM_dict += calculate_ssim(img_groudtruth, img_original)
    if i%10 < 5:
        PNSR_dict_in += calculate_psnr(img_groudtruth, img_original)
        SSIM_dict_in += calculate_ssim(img_groudtruth, img_original)
    else:
        PNSR_dict_out += calculate_psnr(img_groudtruth, img_original)
        SSIM_dict_out += calculate_ssim(img_groudtruth, img_original)

print("PNSR_dict:", PNSR_dict/len(original), ", SSIM_dict:", SSIM_dict/len(original))
print("PNSR_dict_in:", PNSR_dict_in/len(original)*2, ", SSIM_dict_in:", SSIM_dict_in/len(original)*2)
print("PNSR_dict_out:", PNSR_dict_out/len(original)*2, ", SSIM_dict_out:", SSIM_dict_out/len(original)*2)

100%|██████████████████████████████████████████████████████████████████████████████| 1600/1600 [01:39<00:00, 16.16it/s]

PNSR_dict: 15.44735212964792 , SSIM_dict: 0.285619528392657
PNSR_dict_in: 15.657911585267643 , SSIM_dict_in: 0.2944340713775102
PNSR_dict_out: 15.236792674028193 , SSIM_dict_out: 0.27680498540780485
